## A Restaurant Recommendation System based on Yelp Information ##

In [1]:
import re
from threading import Thread
import pandas as pd

**The following cell is the code for recommendation engine:**

In [2]:
def ask_for_name(df):
    
    nl = []
    while True:
        print('Please enter a restaurant name below, if you have no idea about it, just click return\n')
        name=input("Restaurant name: \n")
        
        if name == '':
            return df
        
        for item in df['restaurant_name'].tolist():
            if isinstance(item, str):
                if re.search(name.lower(), item.lower()):
                    nl.append(item)
        
        if len(nl) == 0:
            print('Please enter a valid name.\n')
            continue
        else:
            break
    return df.loc[df['restaurant_name'].isin(nl)]

def ask_for_category(df):
    
    cl = []
    while True:
        print('Please enter a restaurant category below, if you have no idea about it, just click return\n')
        cat = input("Restaurant category: \n")
        
        if cat == '':
            return df
        
        for item in df['Category'].tolist():
            if isinstance(item, str):
                if re.search(cat.lower(), item.lower()):
                    cl.append(item)
        
        if len(cl) == 0:
            print('Please enter a valid category.\n')
            continue
        else:
            break
    return df.loc[df['Category'].isin(cl)]

def ask_for_neighbor(df):
    
    nl = []
    while True:
        print('Please enter a restaurant neighborhood below, if you have no idea about it, just click return\n')
        neighbor = input("Restaurant neighborhood: \n")
        
        if neighbor == '':
            return df
        
        for item in df['restaurant_neighborhood'].tolist():
            if isinstance(item, str):
                if re.search(neighbor.lower(), item.lower()):
                    nl.append(item)
                
        if len(nl) == 0:
            print("Please enter a valid neighborhood. \n")
            continue
        else:
            break
    
    return df.loc[df['restaurant_neighborhood'].isin(nl)]

def ask_for_rating(df):
    
    
    while True:
        try:
            print("Please enter a range of your expected rating below, if you have no idea about it, just click return\n")
            lb = input("Please enter a lower bound of rating range you are looking for :\n")
            ub = input("Please enter an upper bound of rating range you are looking for :\n")
        
            if lb == '' and ub == '':
                return df
        
            if lb == '':
                lb = '0'
        
            if ub == '':
                ub = '5'
            
            if float(lb) > float(ub):
                print('Lower bound has to be smaller than the upper bound.\n')
                continue
        
            if float(lb) < 0 or float(ub) > 5:
                print('The range of rating has to be between 0 and 5.\n')
                continue
            
            return df.loc[(df['restaurant_rating'] >= float(lb)) & (df['restaurant_rating'] <= float(ub))]
        
        except:
        
            print('Please provide a valid input\n')
            continue

# Credit to ZiYuan Wang
def ask_for_party(df):
    
    while True:
        print("Good for groups / parties?\n")
        PARTY = input("Please enter Y/N\n")
        
        if PARTY == 'Y':
            A = 'Yes'
        
        elif PARTY =='N':
            A='No'
            
        else: 
            continue
        
        return df.loc[df['Good for Groups']==A]
    
def ask_for_price(df):
    
    pl = []
    while True:
        try:
            print("Please enter a range of your expected price below, if you have no idea about it, just click return\n")
            lb = input("Please enter a lower bound of price range you are looking for :\n")
            ub = input("Please enter an upper bound of price range you are looking for :\n")
            if lb == '' and ub == '':
                return df
        
            for item in df['price_range'].tolist():
                if isinstance(item, str):
                    if re.search('^\$\d+-\d+$', item):
                        i1 = int(item[1:item.find('-')])
                        i2 = int(item[item.find('-')+1:])
                    
                        if float(lb) <= i1 or float(ub) >= i2:
                            pl.append(item)
                    elif 'Under' in item:
                    
                        if int(item[item.find('$')+1:]) >= float(lb):
                            pl.append(item)
                        
                    elif 'Above' in item:
                    
                        if int(item[item.find('$')+1:]) <= float(ub):
                            pl.append(item)
                        
            if len(pl) == 0:
                print("Unfortunately, there are no restaurants falling into your desired price range. Please try some other price range\n")
                continue
            else:
                
                return df.loc[df['price_range'].isin(pl)]
    
        except:
        
            print('Please provide a valid input\n')
            continue

** Below is three demos for how to use the recommendation engine:**

In [ ]:
# demo 1
global_df = pd.read_csv('final_df.csv', encoding='latin1')
done = False

while done == False:
    
    global_df = ask_for_name(global_df)
    length = len(global_df['restaurant_name'].tolist())
    print(f"There are {length} restaurants filtered out so far\n")
    con = input('Do you want to keep searching? Enter 0 if you are done with searching. Otherwise, enter anything else.\n')
    if con == '0':
        break
    
    while True:
        global_df = ask_for_category(global_df)
        length = len(global_df['restaurant_name'].tolist())
        print(f"There are {length} restaurants filtered out so far\n")
        con = input('Do you want to keep filtering based on Category? Enter 0 if you are done with searching. Otherwise, enter anything else.\n')
        if con == '0':
            break
        
    length = len(global_df['restaurant_name'].tolist())
    print(f"There are {length} restaurants filtered out so far\n")
    con = input('Do you want to keep searching? Enter 0 if you are done with searching. Otherwise, enter anything else.\n')
    if con == '0':
        break
        
    while True:
        global_df = ask_for_neighbor(global_df)
        length = len(global_df['restaurant_name'].tolist())
        print(f"There are {length} restaurants filtered out so far\n")
        con = input('Do you want to keep filtering based on Neighborhood? Enter 0 if you are done with searching. Otherwise, enter anything else.\n')
        if con == '0':
            break
    
    length = len(global_df['restaurant_name'].tolist())
    print(f"There are {length} restaurants filtered out so far\n")
    con = input('Do you want to keep searching? Enter 0 if you are done with searching. Otherwise, enter anything else.\n')
    if con == '0':
        break
    
    global_df = ask_for_rating(global_df)
    length = len(global_df['restaurant_name'].tolist())
    print(f"There are {length} restaurants filtered out so far\n")
    con = input('Do you want to keep searching? Enter 0 if you are done with searching. Otherwise, enter anything else.\n')
    if con == '0':
        break
    
    global_df = ask_for_price(global_df)
    length = len(global_df['restaurant_name'].tolist())
    print(f"There are {length} restaurants filtered out so far\n")
    con = input('Do you want to keep searching? Enter 0 if you are done with searching. Otherwise, enter anything else.\n')
    if con == '0':
        break
        
    global_df = ask_for_party(global_df)
    length = len(global_df['restaurant_name'].tolist())
    print(f"There are {length} restaurants filtered out so far\n")
    con = input('Do you want to keep searching? Enter 0 if you are done with searching. Otherwise, enter anything else.\n')
    if con == '0':
        break
        
    done = True
    
a4 = global_df
global_df

In [ ]:
def get_out_put_1(d,f):
    print(f'''###Recommendation{i+1}###''')
    print(f'''restaurant name:                {d.iloc[i,32]}''')
    print(f'''restaurant address:             {d.iloc[i,37]} New York, NY {d.iloc[0,36]}''')
    print(f'''price range:                        {d.iloc[i,31]}''') 
    print(f'''restaurant rating:                {d.iloc[i,34]}''')
    print(f'''restaurant hygiene rate:       {d.iloc[i,20]}''')
    print(f'''restaurant attire:                 {d.iloc[i,5]} ''' )
    print(f'''restaurant parking:              {d.iloc[i,25]}''')
    print(f'''restaurant reservable:          {d.iloc[i,26]}''')
    print(f'''restaurant ambience:           {d.iloc[i,4]}''')

In [ ]:
def get_out_put_2(d,c,g):
    print(f'''###Recommendation{i+1}###''')
    print(f'''restaurant name:                {d.iloc[a[i],32]}''')
    print(f'''restaurant address:             {d.iloc[a[i],37]} New York, NY {d.iloc[a[i],36]} ''')
    print(f'''price range:                        {d.iloc[a[i],31]}''')
    print(f'''restaurant rating:                {d.iloc[a[i],34]}''')
    print(f'''restaurant hygiene rate:       {d.iloc[a[i],20]}''')
    print(f'''restaurant attire:                 {d.iloc[a[i],5]}''' )
    print(f'''restaurant ambience:           {d.iloc[a[i],4]}''')
    print(f'''restaurant parking:              {d.iloc[a[i],25]}''')
    print(f'''restaurant reservable:          {d.iloc[a[i],26]}''')

In [ ]:
if a4.shape[0]<=5:
    for i in range(a4.shape[0]):
        get_out_put_1(a4,i)
        print('\n')
elif a4.shape[0]>5:
    a=[]
    list_1=[]
    for q in range(a4.shape[0]):
        list_1.append(a4.iloc[q,34])
    list_1s=sorted(list_1, reverse=True)
    for i in range(len(list_1)):
        if list_1s[0]==list_1[i]:
            a.append(i)
            break
    list_1[a[0]]=-1
    for i in range(len(list_1)):
        if list_1s[1]==list_1[i]:
            a.append(i)
            break
    list_1[a[1]]=-1
    for i in range(len(list_1)):
        if list_1s[2]==list_1[i]:
            a.append(i)
            break
    list_1[a[2]]=-1
    for i in range(len(list_1)):
        if list_1s[3]==list_1[i]:
            a.append(i)
            break
    list_1[a[3]]=-1
    for i in range(len(list_1)):
        if list_1s[4]==list_1[i]:
            a.append(i)
            break
    list_1[a[4]]=-1
    for i in range(len(a)):
        get_out_put_2(a4,i,a)
        print('\n')
elif a4.shape[0]<=0:
    print("Please search again, this time we do not have a restaurant satisfy your need. ")
